# Retrieving Days with an orange dot (NASA labeled wildfire), minimal cloud coverage, too much cloud coverage, what I believe is wildfire smoke and what I think is not wildfire smoke

* I check some of the results for just 2018 and it actually did a really good job. Because there isn't much (maybe any) orange color in the images, I am going to assume if this returns a date with orange pixels then there was a wildfire on that date in Redding.
* I could manually go through all the images but that would take a very long time.
* To check, I have the excel files, California_Fire_Incidents_Shasta.csv and California_Fire_Incidents.csv, that have dates of actual wildfires so I can cross check to confirm.
* Note: The algorithm I train on the images will use the images that don't have the orange color (that came from nasa)

* Data was originally from here: https://fsapps.nwcg.gov/afm/imagery.php
* That site is being discontinued for:
    * https://wvs.earthdata.nasa.gov/
    * https://firms.modaps.eosdis.nasa.gov/usfs/
    * https://wvs.earthdata.nasa.gov/?LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,MODIS_Terra_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2020-09-21&COORDINATES=40.200000,-122.500000,41.000000,-121.800000&FORMAT=image/jpeg&AUTOSCALE=TRUE&RESOLUTION=10km

In [33]:
from PIL import Image
from collections import defaultdict
import requests

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imshow, imread

In [1]:
#im = Image.open('images_w_fire_labels/2018-07-31.jpg')

# Just getting images with smoke to see colors

In [36]:
unlabeled_url_full = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor&CRS=EPSG:4326&TIME=2020-08-21&WRAP=DAY&BBOX=40.3,-122.55,40.5,-121.75&FORMAT=image/jpeg&WIDTH=9&HEIGHT=2&AUTOSCALE=TRUE&ts=1642005890751'

In [37]:
unlabeled_url_1 = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor&CRS=EPSG:4326&TIME='
time = '2020-08-21'
unlabeled_url_2 = '&WRAP=DAY&BBOX=40.3,-122.3,40.5,-121.9&FORMAT=image/jpeg&'
url_height_width = 'WIDTH=240&HEIGHT=180'
unlabeled_url_3 = '&AUTOSCALE=TRUE&ts=1642005890751'

# The images have 32760 total pixels

In [38]:
date = time
url = unlabeled_url_1+date+unlabeled_url_2+url_height_width+unlabeled_url_3
r = requests.get(url)
if r.status_code == 200:
    storage_location = 'wildfire_smoke_example/'+date+'v2'+'.jpg'
    with open(storage_location, "wb") as f:
        f.write(r.content)
else:
    print('date failed: ', date)

In [39]:
# (240,240,240) is very close to white
img = Image.new('RGB',(200,200),(220, 220, 220))
img.show()

# actual_wildfire_pixels is a set of (r,g,b) that has wildfire smoke

# 8/25/2020 and 8/21/2020 look like wildfire smoke

In [40]:
im = Image.open('wildfire_smoke_example/2020-08-21v2.jpg')
by_color = defaultdict(int)

actual_wildfire_pixels = set()
for pixel in im.getdata():
    by_color[pixel] += 1
    actual_wildfire_pixels.add(pixel)

# actual_wildfire_pixels range from (137,143,141) to (177,174,169)

In [42]:
# This is the first rgb value for each of the 180 rows
# This returns 180 lists with 180 elements
#im[:,:,0]
len(im[:,:,0])

180

# All this does is put a 0 if the pixels are less than the filter
[[229 220 217 ... 202 193 189]
 [223 214 211 ... 210 200 193]
 [220 212 208 ... 215 203 193]
 ...
 [  0 167 194 ... 225 220 212]
 [179   0   0 ... 223 218 211]
 [186   0   0 ... 221 216 210]]

In [43]:
wildfire_smoke_dates_test = set()
no_wildfire_smoke_dates_test = set()
# 2020-10-23 has no clouds in it
#date = '2020-10-23'
date = '2021-01-24'
storage_location = 'images_wo_fire_labels/'+date+'.jpg'
im = imread(storage_location)
wildfire_smoke_filtered = (im[:,:,0] >= 160) & (im[:,:,1] >= 165) & (im[:,:,2] >= 160)
im[:, :, 0] = im[:, :, 0] * wildfire_smoke_filtered
print(len(im[:, :, 0]))
#print(im[:, :, 0])
im[:, :, 1] = im[:, :, 1] * wildfire_smoke_filtered
print(len(im[:, :, 1]))
#print(im[:, :, 1])
im[:, :, 2] = im[:, :, 2] * wildfire_smoke_filtered
print(len(im[:, :, 2]))
#print(im[:, :, 2])
                
wildfire_smoke_count = 0

# Also have to do im[:,:,1] and im[:,:,2]
for pixel in im[:,:,0]:#[:,:,0]:
    for point in pixel:
        #print('point: ', point)
        break
    break
        #if point > 0:
            #wildfire_smoke_count += 1
if wildfire_smoke_count > 0:
    wildfire_smoke_dates_test.add(date)
else:
    no_wildfire_smoke_dates_test.add(date)
print(wildfire_smoke_count)

180
180
180
0


In [44]:
unlabeled_url_1 = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor&CRS=EPSG:4326&TIME='
date_format = '2020-08-21'
unlabeled_url_2 = '&WRAP=DAY&BBOX=40,-123,41.15,-121&FORMAT=image/jpeg&'
url_height_width = 'WIDTH=240&HEIGHT=180'
unlabeled_url_3 = '&AUTOSCALE=TRUE&ts=1642005890751'

In [45]:
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
days = ['01','02','03','04','05','06','07','08','09','10',
       '11','12','13','14','15','16','17','18','19','20',
       '21','22','23','24','25','26','27','28','29',
       '30','31']

In [46]:
days_31 = days[0:31]
days_30 = days[0:30]
days_29 = days[0:29]
days_28 = days[0:28]

# Images with Orange Dots

In [61]:
orange_fire_dates = set()
no_orange_fire_dates = set()
all_dates = {}
for year in years:
    for month in months:
        if (month == '01' or month == '03' or month == '05' or month == '07' or month == '08' or month == '10' or month == '12') and (year == '2010' or year == '2011' or year == '2012' or year == '2013' or year == '2014' or year == '2015' or year == '2016' or year == '2017' or year == '2018' or year == '2019' or year == '2020' or year == '2021'):
            for day in days_31:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im[:,:,0]:
                    for point in pixel:
                        if point > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates.add(date)
                    all_dates[date] = 0
        if (month == '04' or month == '06' or month == '09' or month == '11') and (year == '2010' or year == '2011' or year == '2012' or year == '2013' or year == '2014' or year == '2015' or year == '2016' or year == '2017' or year == '2018' or year == '2019' or year == '2020' or year == '2021'):
            for day in days_30:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im[:,:,0]:
                    for point in pixel:
                        if point > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates.add(date)
                    all_dates[date] = 0
        if month == '02' and (year == '2010' or year == '2011' or year == '2013' or year == '2014' or year == '2015' or year == '2017' or year == '2018' or year == '2019' or year == '2021'):
            for day in days_28:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im[:,:,0]:
                    for point in pixel:
                        if point > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates.add(date)
                    all_dates[date] = 0
        if month == '02' and (year == '2012' or year == '2016' or year == '2020'):
            for day in days_29:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im[:,:,0]:
                    for point in pixel:
                        if point > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates.add(date)
                    all_dates[date] = 0

In [62]:
len(orange_fire_dates)

324

In [63]:
len(no_orange_fire_dates)

4059

# Orange dot all pixels
# The dates in orange_fire_dates_all_pixels that aren't in orange_fire_dates are just black images
* Can drop those dates when trying to train

In [77]:
orange_fire_dates_all_pixels = set()
no_orange_fire_dates_all_pixels = set()
all_dates = {}
for year in years:
    for month in months:
        if (month == '01' or month == '03' or month == '05' or month == '07' or month == '08' or month == '10' or month == '12') and (year == '2010' or year == '2011' or year == '2012' or year == '2013' or year == '2014' or year == '2015' or year == '2016' or year == '2017' or year == '2018' or year == '2019' or year == '2020' or year == '2021'):
            for day in days_31:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im:
                    for point in pixel:
                        if point.any() > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 0
        if (month == '04' or month == '06' or month == '09' or month == '11') and (year == '2010' or year == '2011' or year == '2012' or year == '2013' or year == '2014' or year == '2015' or year == '2016' or year == '2017' or year == '2018' or year == '2019' or year == '2020' or year == '2021'):
            for day in days_30:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im:
                    for point in pixel:
                        if point.any() > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 0
        if month == '02' and (year == '2010' or year == '2011' or year == '2013' or year == '2014' or year == '2015' or year == '2017' or year == '2018' or year == '2019' or year == '2021'):
            for day in days_28:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im:
                    for point in pixel:
                        if point.any() > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 0
        if month == '02' and (year == '2012' or year == '2016' or year == '2020'):
            for day in days_29:
                date = year+'-'+month+'-'+day
                storage_location = 'images_w_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                red_filtered = (im[:,:,0] > 150) & (im[:,:,1] < 100) & (im[:,:,2] < 110)
                im[:, :, 0] = im[:, :, 0] * red_filtered
                im[:, :, 1] = im[:, :, 1] * red_filtered
                im[:, :, 2] = im[:, :, 2] * red_filtered
                orange_count = 0
                by_color = defaultdict(int)
                for pixel in im:
                    for point in pixel:
                        if point.any() > 0:
                            orange_count += 1
                if orange_count > 0:
                    orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 1
                else:
                    no_orange_fire_dates_all_pixels.add(date)
                    all_dates[date] = 0

In [78]:
print(len(orange_fire_dates_all_pixels))
print(len(no_orange_fire_dates_all_pixels))

338
4045


# Looking at Smoke pixels and cloud pixels

In [65]:
fire_dates = set()
no_fire_dates = set()
cloudy_dates = set()
no_cloudy_dates = set()
for year in years:
    for month in months:
        if (month == '01' or month == '03' or month == '05' or month == '07' or month == '08' or month == '10' or month == '12') and (year == '2010' or year == '2011' or year == '2012' or year == '2013' or year == '2014' or year == '2015' or year == '2016' or year == '2017' or year == '2018' or year == '2019' or year == '2020' or year == '2021'):
            for day in days_31:
                date = year+'-'+month+'-'+day
                storage_location = 'images_wo_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                fire = 0
                no_fire = 0
                cloud = 0
                no_cloud = 0
                for pixel in im:
                    for point in pixel:
                        if point[0] <= 177 and point[0] > 137 and point[1] < 174 and point[1] > 143 and point[2] < 169 and point[2] > 141:
                            fire += 1
                        else:
                            no_fire +=1  
                        if point[0] >= 220 and point[1] >= 220 and point[2] >= 220:
                            cloud += 1
                        else:
                            no_cloud+=1
                    
                percent_fire = fire/32760
                if percent_fire >=.15:
                    fire_dates.add(date)
                percent_no_fire = no_fire/32760
                if percent_no_fire >=.85:
                    no_fire_dates.add(date)
                percent_cloud = cloud/32760
                if percent_cloud >=.3:
                    cloudy_dates.add(date)
                percent_no_cloud = no_cloud/32760
                if percent_no_cloud >= .9:
                    no_cloudy_dates.add(date)
                    
        if (month == '04' or month == '06' or month == '09' or month == '11') and (year == '2010' or year == '2011' or year == '2012' or year == '2013' or year == '2014' or year == '2015' or year == '2016' or year == '2017' or year == '2018' or year == '2019' or year == '2020' or year == '2021'):
            for day in days_30:
                date = year+'-'+month+'-'+day
                storage_location = 'images_wo_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                fire = 0
                no_fire = 0
                cloud = 0
                no_cloud = 0
                for pixel in im:
                    for point in pixel:
                        if point[0] <= 177 and point[0] > 137 and point[1] < 174 and point[1] > 143 and point[2] < 169 and point[2] > 141:
                            fire += 1
                        else:
                            no_fire +=1  
                        if point[0] >= 220 and point[1] >= 220 and point[2] >= 220:
                            cloud += 1
                        else:
                            no_cloud+=1
                    
                percent_fire = fire/32760
                if percent_fire >=.15:
                    fire_dates.add(date)
                percent_no_fire = no_fire/32760
                if percent_no_fire >=.85:
                    no_fire_dates.add(date)
                percent_cloud = cloud/32760
                if percent_cloud >=.3:
                    cloudy_dates.add(date)
                percent_no_cloud = no_cloud/32760
                if percent_no_cloud >= .9:
                    no_cloudy_dates.add(date)      
                    
        if month == '02' and (year == '2010' or year == '2011' or year == '2013' or year == '2014' or year == '2015' or year == '2017' or year == '2018' or year == '2019' or year == '2021'):
            for day in days_28:
                date = year+'-'+month+'-'+day
                storage_location = 'images_wo_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                fire = 0
                no_fire = 0
                cloud = 0
                no_cloud = 0
                for pixel in im:
                    for point in pixel:
                        if point[0] <= 177 and point[0] > 137 and point[1] < 174 and point[1] > 143 and point[2] < 169 and point[2] > 141:
                            fire += 1
                        else:
                            no_fire +=1  
                        if point[0] >= 220 and point[1] >= 220 and point[2] >= 220:
                            cloud += 1
                        else:
                            no_cloud+=1
                    
                percent_fire = fire/32760
                if percent_fire >=.15:
                    fire_dates.add(date)
                percent_no_fire = no_fire/32760
                if percent_no_fire >=.85:
                    no_fire_dates.add(date)
                percent_cloud = cloud/32760
                if percent_cloud >=.3:
                    cloudy_dates.add(date)
                percent_no_cloud = no_cloud/32760
                if percent_no_cloud >= .9:
                    no_cloudy_dates.add(date)
                           
        if month == '02' and (year == '2012' or year == '2016' or year == '2020'):
            for day in days_29:
                date = year+'-'+month+'-'+day
                storage_location = 'images_wo_fire_labels/'+date+'.jpg'
                im = imread(storage_location)
                fire = 0
                no_fire = 0
                cloud = 0
                no_cloud = 0
                for pixel in im:
                    for point in pixel:
                        if point[0] <= 177 and point[0] > 137 and point[1] < 174 and point[1] > 143 and point[2] < 169 and point[2] > 141:
                            fire += 1
                        else:
                            no_fire +=1  
                        if point[0] >= 220 and point[1] >= 220 and point[2] >= 220:
                            cloud += 1
                        else:
                            no_cloud+=1
                    
                percent_fire = fire/32760
                if percent_fire >=.15:
                    fire_dates.add(date)
                percent_no_fire = no_fire/32760
                if percent_no_fire >=.85:
                    no_fire_dates.add(date)
                percent_cloud = cloud/32760
                if percent_cloud >=.3:
                    cloudy_dates.add(date)
                percent_no_cloud = no_cloud/32760
                if percent_no_cloud >= .9:
                    no_cloudy_dates.add(date)

In [67]:
print(len(fire_dates))
print(len(no_fire_dates))
print(len(cloudy_dates))
print(len(no_cloudy_dates))

301
4082
765
3249


In [52]:
import pandas as pd

# Have only orange fire dates
* orange_fire_dates
* no_orange_fire_dates

# Have only orange fire dates but also images that are just black
* orange_fire_dates_all_pixels
* no_orange_fire_dates_all_pixels

# Have dates where wildfire smoke was detected
* fire_dates

# Have daes where no wildfire smoke was detected (or not a lot)
* no_fire_dates

# Have dates with too much cloud coverage
* cloudy_dates

# Have dates that are clear
* no_cloudy_dates

In [89]:
orange_fire_dates_list = list(orange_fire_dates)

In [90]:
orange_fire_dates_list_df = pd.DataFrame(orange_fire_dates_list,columns = ['Date'])

In [94]:
orange_fire_dates_list_df['Fire'] = 1

In [96]:
orange_fire_dates_list_df.to_csv('orange_fire_dates_1_13_2022.csv')

In [97]:
no_orange_fire_dates_list = list(no_orange_fire_dates)

In [98]:
no_orange_fire_dates_list_df = pd.DataFrame(no_orange_fire_dates_list,columns = ['Date'])

In [99]:
no_orange_fire_dates_list_df['Fire'] = 0

In [100]:
no_orange_fire_dates_list_df.to_csv('no_orange_fire_dates_1_13_2022.csv')

In [101]:
orange_fire_dates_all_pixels_list = list(orange_fire_dates_all_pixels)

In [102]:
orange_fire_dates_all_pixels_list_df = pd.DataFrame(orange_fire_dates_all_pixels_list,columns = ['Date'])

In [103]:
orange_fire_dates_all_pixels_list_df['Fire'] = 1

In [104]:
orange_fire_dates_all_pixels_list_df.to_csv('orange_fire_dates_all_pixels_1_13_2022.csv')

In [105]:
no_orange_fire_dates_all_pixels_list = list(no_orange_fire_dates_all_pixels)

In [106]:
no_orange_fire_dates_all_pixels_list_df = pd.DataFrame(no_orange_fire_dates_all_pixels_list,columns = ['Date'])

In [107]:
no_orange_fire_dates_all_pixels_list_df['Fire'] = 0

In [108]:
no_orange_fire_dates_all_pixels_list_df.to_csv('no_orange_fire_dates_all_pixels_1_13_2022.csv')

In [ ]:
# Wildfire smoke

In [109]:
wildfire_smoke_detected_list = list(fire_dates)

In [110]:
wildfire_smoke_detected_list_df = pd.DataFrame(wildfire_smoke_detected_list,columns = ['Date'])

In [111]:
wildfire_smoke_detected_list_df['Fire'] = 1

In [112]:
wildfire_smoke_detected_list_df.to_csv('wildfire_smoke_detected_1_13_2022.csv')

In [113]:
# No wildfire smoke

In [114]:
no_wildfire_smoke_detected_list = list(no_fire_dates)

In [115]:
no_wildfire_smoke_detected_list_df = pd.DataFrame(no_wildfire_smoke_detected_list,columns = ['Date'])

In [116]:
no_wildfire_smoke_detected_list_df['Fire'] = 0

In [117]:
no_wildfire_smoke_detected_list_df.to_csv('no_wildfire_smoke_detected_1_13_2022.csv')

In [ ]:
# Clouds

In [118]:
cloudy_dates_list = list(cloudy_dates)

In [119]:
cloudy_dates_list_df = pd.DataFrame(cloudy_dates_list,columns = ['Date'])

In [120]:
cloudy_dates_list_df['Clouds'] = 1

In [121]:
cloudy_dates_list_df.to_csv('cloudy_dates_list_1_13_2022.csv')

In [ ]:
# No Clouds

In [122]:
no_cloudy_dates_list = list(no_cloudy_dates)

In [123]:
no_cloudy_dates_list_df = pd.DataFrame(no_cloudy_dates_list,columns = ['Date'])

In [124]:
no_cloudy_dates_list_df['Clouds'] = 0

In [125]:
no_cloudy_dates_list_df.to_csv('no_cloudy_dates_list_1_13_2022.csv')